In [64]:
import pandas as pd
import numpy as np

import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import string

from tashaphyne.stemming import ArabicLightStemmer
import arabicstopwords.arabicstopwords as stp
from ar_corrector.corrector import Corrector
import re

import nltk
from nltk.corpus import stopwords
import wordcloud
from nltk.tokenize import word_tokenize
from nltk.stem.isri import ISRIStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.cluster import DBSCAN, SpectralClustering, MeanShift
from sklearn.preprocessing import StandardScaler

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [65]:
df=pd.read_csv('new.csv')
df2=pd.read_csv('try.csv')

In [66]:
pd.set_option('display.max_columns', 30)

In [67]:
r = [df, df2]
df = pd.concat(r)
df.drop_duplicates(inplace=True)

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 351 entries, 0 to 137
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   links          351 non-null    object
 1   court          351 non-null    object
 2   court_id       351 non-null    object
 3   court_date     351 non-null    object
 4   appeal_court   307 non-null    object
 5   appeal_id      307 non-null    object
 6   appeal_date    274 non-null    object
 7   region         274 non-null    object
 8   city           318 non-null    object
 9   judgment_text  351 non-null    object
 10  appeal_text    274 non-null    object
 11  appeal_data    33 non-null     object
 12  appleal_text   33 non-null     object
dtypes: object(13)
memory usage: 38.4+ KB


In [69]:
df.duplicated().sum()

0

In [70]:
df_Train, df_test = train_test_split(df, test_size=0.2, random_state= 30)

df_train, df_val = train_test_split(df_Train, test_size=0.2, random_state= 30)

In [71]:
print(f'Shape of train:', df_train.shape)
print(f'Shape of validation:', df_val.shape)
print(f'Shape of test:', df_test.shape)

Shape of train: (224, 13)
Shape of validation: (56, 13)
Shape of test: (71, 13)


In [72]:
df_train.isnull().sum()

links              0
court              0
court_id           0
court_date         0
appeal_court      26
appeal_id         26
appeal_date       45
region            45
city              19
judgment_text      0
appeal_text       45
appeal_data      205
appleal_text     205
dtype: int64

In [73]:
df_val.isnull().sum()

links             0
court             0
court_id          0
court_date        0
appeal_court      4
appeal_id         4
appeal_date      12
region           12
city              8
judgment_text     0
appeal_text      12
appeal_data      48
appleal_text     48
dtype: int64

In [74]:
df_train.appeal_court.value_counts(),

(محكمة الاستئناف: المحكمة التجارية    96
 المحكمة التجارية                     87
 محكمة الاستئناف: محكمة الاستئناف      8
 محكمة الاستئناف                       7
 Name: appeal_court, dtype: int64,)

In [75]:
df_val.appeal_court.value_counts(),

(محكمة الاستئناف: المحكمة التجارية    30
 المحكمة التجارية                     21
 محكمة الاستئناف                       1
 Name: appeal_court, dtype: int64,)

In [76]:
df_train['appeal_court'].fillna(df_train['appeal_court'].mode()[0], inplace = True)

In [77]:
df_val['appeal_court'].fillna(df_val['appeal_court'].mode()[0], inplace = True)

In [78]:
df_train.appeal_id.value_counts()

 ٣٢٣               2
 ٦٥٤               2
 ٢٦٣٠              2
 ٦٨٦               2
 ٧٦                2
                  ..
 ٤٢٦               1
 ٧٥                1
رقم القرار: ١٥٨    1
 ٥٤٤٩              1
 ٣٠                1
Name: appeal_id, Length: 192, dtype: int64

In [79]:
df_val.appeal_id.value_counts()

رقم القرار: ٦٨٨     1
رقم القرار: ٥٢٥٣    1
رقم القرار: ٥٣٠٠    1
رقم القرار: ٥٤٩٢    1
رقم القرار: ٤٧١٦    1
رقم القرار: ٥٤٩١    1
 ٥٤٢                1
 ١٤٥                1
رقم القرار: ٤٧٩٥    1
 ٢٢                 1
رقم القرار: ٤١٧١    1
 ٥٩١                1
رقم القرار: ٧٩٥     1
 ٧٣                 1
 ٣٦                 1
 ٣٧٥                1
 ٤٤٢١               1
 ٤٤٣٦               1
رقم القرار: ٥٤٧٠    1
رقم القرار: ٣٧٥     1
رقم القرار: ٥٢٦٧    1
 ٢٨٩٣               1
 ٣٤٣٧               1
 ٥٤٩١               1
رقم القرار: ٩٣      1
 ٥٤٧٧               1
رقم القرار: ٢٣٠٧    1
رقم القرار: ٢       1
رقم القرار: ٥٢٨١    1
 ٧٠٩                1
رقم القرار: ٩٦      1
رقم القرار: ٤٢٦     1
رقم القرار: ٣٨٠٣    1
رقم القرار: ٥٣٤٣    1
رقم القرار: ٣٢٣     1
رقم القرار: ٥٩٨     1
 ٥٣٨١               1
 ٥٥٨١               1
 ٧٥٧                1
رقم القرار: ٥١٣٣    1
رقم القرار: ٢١٥٧    1
 ٥٢٦٧               1
رقم القرار: ٣٤٥٤    1
رقم القرار: ٥٤٠٣    1
 ٥٦٣٦               1
 ٩٦       

In [80]:
df_train.dropna(subset=['appeal_id'], inplace=True);

In [81]:
df_val.dropna(subset=['appeal_id'], inplace=True);

In [82]:
df_train['appeal_date'] = df_train['appeal_date'].str.replace('تاريخه:','')

In [83]:
df_val['appeal_date'] = df_val['appeal_date'].str.replace('تاريخه:','')

In [84]:
df_train['appeal_id'] = df_train['appeal_id'].str.replace('رقم القرار:','')

In [85]:
df_val['appeal_id'] = df_val['appeal_id'].str.replace('رقم القرار:','')

In [86]:
df_val['appeal_date'] = df_val['appeal_id'].str.replace('رقم القرار:','')

In [87]:
df_train['court'] = df_train['court'].str.replace('المحكمة:','')

In [88]:
df_train['court_id'] = df_train['court_id'].str.replace('رقم القضية - القرار:','')

In [89]:
df_val['court_id'] = df_val['court_id'].str.replace('رقم القضية - القرار:','')

In [90]:
df_train['court_date'] = df_train['court_date'].str.replace('تاريخها: ','')

In [91]:
df_val['court_date'] = df_val['court_date'].str.replace('تاريخها: ','')

In [92]:
df_train['appeal_court'] = df_train['appeal_court'].str.replace('محكمة الاستئناف: ','')

In [93]:
df_val['appeal_court'] = df_val['appeal_court'].str.replace('محكمة الاستئناف: ','')

In [94]:
df_train['region'] = df_train['region'].str.replace('المدينة: ','')

In [95]:
df_val['region'] = df_val['region'].str.replace('المدينة: ','')

In [96]:
df_train['city'] = df_train['city'].str.replace('المدينه: ','')

In [97]:
df_val['city'] = df_val['city'].str.replace('المدينه: ','')

In [98]:
df_train.city.value_counts()

الرياض             148
الدمام              14
المدينة المنورة     12
بريدة                5
Name: city, dtype: int64

In [99]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations
df_train.judgment_text = df_train.judgment_text.apply(lambda x: x.translate(str.maketrans('', '', punctuations_list)))
# df_train.appeal_text = df_train.appeal_text.apply(lambda x: x.translate(str.maketrans('', '', punctuations_list)))

In [100]:
df_train.judgment_text = df_train.judgment_text.apply(lambda x: x.translate(str.maketrans('', '', punctuations_list)))
df_val.judgment_text = df_val.judgment_text.apply(lambda x: x.translate(str.maketrans('', '', punctuations_list)))
# df_val.appeal_text = df_val.appeal_text.apply(lambda x: x.translate(str.maketrans('', '', punctuations_list)))

In [101]:
df_train

,links,court,court_id,court_date,appeal_court,appeal_id,appeal_date,region,city,judgment_text,appeal_text,appeal_data,appleal_text
86,القضية رقم ٦٨١٧ لعام ١٤٤٢ هـ,المحكمة التجارية,٦٨١٧,٢٥/١١/١٤٤٢,المحكمة التجارية,٦٨٦,٦/٣/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام رسول ﷲ الدائرة التجا...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN
33,القضية رقم ٥٨٥٣ لعام ١٤٤٢ هـ,المحكمة التجارية,٥٨٥٣,٢١/٩/١٤٤٢,المحكمة التجارية,٥٢٠٥,٦/٣/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN
39,القضية رقم ٦٨١٧ لعام ١٤٤٢ هـ,المحكمة التجارية,٦٨١٧,٢٥/١١/١٤٤٢,المحكمة التجارية,٦٨٦,٦/٣/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN
46,القضية رقم ٤٤٣١ لعام ١٤٣٩ هـ,المحكمة التجارية,٤٤٣١,١٤/٤/١٤٤١,المحكمة التجارية,٣٥٨٢,٨/٢/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام رسول ﷲ الدائرة التجا...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN
84,القضية رقم ٦٤٤٦ لعام ١٤٣٩ هـ,المحكمة التجارية,٦٤٤٦,٣٠/١/١٤٤١,المحكمة التجارية,٣٦٨٣,٨/٢/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام رسول ﷲ الدائرة التجا...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,القضية رقم ١٦٥٦٣ لعام ١٤٤٠ هـ,المحكمة التجارية,١٦٥٦٣,١٨/٣/١٤٤٢,المحكمة التجارية,٢٩٨٥,٥/٣/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN
46,القضية رقم ٣١٨٧ لعام ١٤٤٢ هـ,المحكمة التجارية,٣١٨٧,٢٣/١٢/١٤٤٢,المحكمة التجارية,٥٩١,٦/٣/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN
26,القضية رقم ٧٤٥٧ لعام ١٤٣٩ هـ,المحكمة التجارية,٧٤٥٧,٩/٩/١٤٤٢,المحكمة التجارية,٥٢٥٣,٨/٢/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام رسول ﷲ الدائرة التجا...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN
95,القضية رقم ١٨٥٩ لعام ١٤٤١ هـ,المحكمة التجارية,١٨٥٩,١٨/١١/١٤٤٢,محكمة الاستئناف,١١٨,٧/٢/١٤٤٣,المنطقة الشرقية,الدمام,الحمد لله والصلاة والسلام رسول ﷲ الدائرة التجا...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN


# Preprocessing

### Remove stopwords

In [103]:
df_train['judgment_text'] = df_train['judgment_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stp.stopwords_list()]))
# df_train['appeal_text'] = df_train['appeal_text'].apply(lambda y: ' '.join([word for word in y.split() if word not in stp.stopwords_list()]))


### Correct words

In [104]:
corr = Corrector()
# df_train['judgment_text'] = df_train['judgment_text'].apply(lambda x: corr.spell_correct(x))

### Remove digits

In [105]:
df_train['judgment_text'] = df_train['judgment_text'].apply(lambda x: re.sub('\w*\d\w*', '', x))
# df_train['appeal_text'] = df_train['appeal_text'].apply(lambda x: re.sub('\w*\d\w*', '', x))

In [106]:
df_train['judgment_text'] = df_train['judgment_text'].str.replace('\n','')

In [107]:
ArListem = ArabicLightStemmer()
ArListem.light_stem('منجزين')

'منجز'

In [108]:
ISRIStemmer().suf32("منجزين")

'منجز'

In [109]:
df_train['judgment_text'] = df_train['judgment_text'].apply(lambda x: ArListem.light_stem(x))
df_train['judgment_text'] = df_train['judgment_text'].apply(lambda x: ISRIStemmer().suf32(x))


In [110]:
# corr.spell_correct(df_train['judgment_text'][0])

In [111]:
df_train['judgment_text'][0]

'الحمد لله والصلاة والسلام رسول ﷲ الدائرة التجارية الثانية والعشرون وبناء القضية رقم  لعام  المقامة عبدﷲ ابراهيم المحمود سجل تجاري ضد تغريد بنت محمد صفير هوية وطنية القاضي عمر مبارك الدوسري رئيسا الوقائع تتلخص وقائع الدعوى وبالقدر اللازم لإصدار الحكم وردت المحكمة التجارية لائحة دعوى مقدمة المدعي وكالة ضد المدعى والمثبتة بياناتها ملف القضية جاء لائحة الدعوى اتفق موكلي الجهة المدعى بتأريخ   لتنفيذ أعمال الموقع الخارجي لمشروع القوات البحرية الملكية السعودية الجبيل وتمثل  أعمال المشروع والمتمثلة أعمال السفلتة وغرف التفتيش وعددها  غرفة ومدة العقد  خمسة عشر يوما تنجز الجهة المدعى  الأعمال وعدد  غرف تفتيش عدد  غرفة واستلموا كامل المبلغ وقدره  مئتان واثنان وعشرون الف ريال إتمام المشروع والمستحق الفعلي  مئة وعشرة ألف ريالا أطلب يلزم الجهة المدعى مؤسسة قمة الصحراء للمقاولات العامة بدفع لموكلتي مبلغ وقدره  مئة واثنا عشر ألف ريالا قيدت الأوراق قضية بالرقم المشار بعالية أحيلت الدائرة فباشرت نظرها النحو الموضح بمحاضر الضبط جلسة يوم  حضر وكيل المدعي بوكالة رقم  ذكر المدعي دعواه مقامة شركة بيت الجزيرة

# remove english words

In [112]:
df_train.judgment_text = df_train.judgment_text.apply(lambda x: re.sub('([@A-Za-z0-9_]+)|[^\w\s]|#|http\S+', '', x) )

# TF-IDF

In [113]:
cv_tfidf = TfidfVectorizer(min_df=2, max_df=20)
x = cv_tfidf.fit_transform(df_train['judgment_text']).toarray()

In [114]:
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [115]:
dt_tfidf = pd.DataFrame(x,columns=cv_tfidf.get_feature_names())



,آب,آت,آثارها,آجل,آخرين,آر,آكد,آلاف,آلة,آلت,آلي,آلية,آمنوا,آن,آنف,...,يوضح,يوفر,يوفق,يوفقكم,يوفي,يوقع,يومنا,يوميا,يومية,يومين,يونيه,يونيو,يوهمني,يي,ييثبت
0,0.0,0.0,0.0,0.115271,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.115271,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.045473,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.028866,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
194,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
195,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.023678,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
196,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## SVD

In [142]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components = 100)
SVD= svd.fit_transform(dt_tfidf)

In [149]:
SVD_DF = pd.DataFrame(SVD)


In [153]:
SVD_DF

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,...,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,0.161024,-0.047824,-0.145822,-0.310299,0.658806,0.428214,0.347849,0.061013,0.064042,0.004118,-0.069471,0.028311,-0.009545,-0.087130,0.107199,...,0.004255,0.000069,0.001237,-0.000715,-0.005658,-0.000865,0.006195,0.003465,0.000122,-0.007583,-0.001465,-0.002474,0.010291,0.007308,0.008623
1,0.244803,0.384457,0.058409,-0.105871,0.012889,-0.008243,0.045806,0.028735,-0.101841,-0.026984,-0.006724,-0.017439,0.020430,-0.038085,0.030269,...,-0.082521,0.081719,-0.036792,-0.126161,0.073941,0.053704,0.026378,0.066285,0.105332,0.072640,-0.128646,0.019456,-0.038038,-0.063467,0.019316
2,0.161024,-0.047824,-0.145822,-0.310299,0.658806,0.428214,0.347849,0.061013,0.064042,0.004118,-0.069471,0.028311,-0.009545,-0.087130,0.107199,...,0.004255,0.000069,0.001237,-0.000715,-0.005658,-0.000865,0.006195,0.003465,0.000122,-0.007583,-0.001465,-0.002474,0.010291,0.007308,0.008623
3,0.152169,-0.055826,-0.031701,-0.014324,-0.011118,-0.031260,-0.029712,-0.047439,-0.002560,0.014830,-0.086638,-0.037576,0.020924,0.032180,-0.046958,...,-0.199494,-0.097921,-0.091739,-0.116114,-0.201857,0.137043,0.177518,0.005516,-0.125382,-0.090284,0.006077,-0.097488,-0.071956,0.159868,0.143697
4,0.182461,-0.014169,-0.059232,0.087785,0.022969,-0.039094,-0.115274,-0.034636,0.232020,-0.062174,0.039976,0.125930,-0.011676,-0.029498,0.085355,...,-0.022015,-0.011054,-0.012945,-0.013731,0.009683,-0.027567,0.024075,-0.006587,-0.010211,-0.000309,0.010108,0.011332,0.013517,0.014685,-0.003803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,0.115803,-0.080616,-0.072715,-0.077268,-0.084966,-0.026970,-0.021815,0.226162,0.025444,0.090971,-0.057892,0.111372,0.074066,0.057272,0.011952,...,-0.011055,0.023986,-0.002455,-0.003206,-0.015210,-0.004232,-0.004662,-0.013452,-0.012912,-0.001912,-0.020604,-0.011103,-0.004518,0.019685,0.007533
194,0.133983,-0.017754,-0.012431,-0.114947,0.243482,0.124566,0.064204,-0.005618,-0.043681,-0.006530,0.019996,0.076966,-0.076341,0.224037,-0.070849,...,0.005370,0.009927,0.003080,-0.018662,0.009848,0.020289,-0.005665,-0.009654,-0.009609,0.016238,0.007096,-0.009784,-0.020648,0.023324,-0.019408
195,0.222573,-0.062777,-0.059993,0.029026,0.036024,-0.052148,-0.110344,-0.123725,-0.047679,-0.117820,0.022841,0.129817,0.088624,-0.018856,0.028405,...,0.126061,0.131602,0.067135,-0.041124,-0.087776,-0.136149,0.031859,-0.136692,0.051519,-0.156577,0.001729,-0.057326,0.081771,0.031003,-0.209680
196,0.112985,-0.029265,-0.052168,0.014483,0.018974,-0.047959,-0.030486,-0.080477,0.029240,-0.093759,0.042641,0.053427,0.111909,-0.076508,0.086045,...,0.018453,-0.018391,0.006484,-0.005730,-0.010969,-0.004794,-0.017983,0.017104,0.011859,0.019687,0.024830,-0.015562,0.005121,-0.003391,0.005517


# Cosine similarity


In [55]:
# df.to_csv('new.csv',index=False)

In [128]:
from numpy import dot
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity


cosine_similarity(SVD_DF.iloc[:,1:2],SVD_DF.iloc[:,2:3])
cosine = lambda v1, v2: dot(v1, v2) / (norm(v1) * norm(v2))


In [151]:
cosine(SVD_DF.iloc[20],SVD_DF.iloc[51])

1.0

In [131]:
x = range(1,197)
y = range(1,197)
for i in x:
    for j in y:
        print(cosine(dt_tfidf.iloc[i],dt_tfidf.iloc[j]))
        print('i= ', i)
        print('j= ', j)
        if cosine(dt_tfidf.iloc[i],dt_tfidf.iloc[j]) > 0.5 and i != j:
            print("---------",cosine(dt_tfidf.iloc[i],dt_tfidf.iloc[j]))
        

1.0
i=  1
j=  1
0.0573774769574995
i=  1
j=  2
0.02009694786403894
i=  1
j=  3
0.008673515872828587
i=  1
j=  4
0.006852093632321315
i=  1
j=  5
0.008220609946818586
i=  1
j=  6
0.027517592271680456
i=  1
j=  7
0.0017362137273743154
i=  1
j=  8
0.019136064830168706
i=  1
j=  9
0.00799062148360443
i=  1
j=  10
0.03148643023100209
i=  1
j=  11
0.02751997773015902
i=  1
j=  12
0.030696600651717074
i=  1
j=  13
0.00775570767638696
i=  1
j=  14
0.009485943639103928
i=  1
j=  15
0.0036621774167898027
i=  1
j=  16
0.013964675323080218
i=  1
j=  17
0.05010909456519788
i=  1
j=  18
0.01859590422080808
i=  1
j=  19
0.018583304048058098
i=  1
j=  20
0.008104975099833971
i=  1
j=  21
0.0
i=  1
j=  22
0.008751718696421388
i=  1
j=  23
0.0
i=  1
j=  24
0.00923187067839965
i=  1
j=  25
0.07570718459701914
i=  1
j=  26
0.008286855283639696
i=  1
j=  27
0.029857494985458467
i=  1
j=  28
0.0024144755175073956
i=  1
j=  29
0.04642357967369461
i=  1
j=  30
0.012652787663622333
i=  1
j=  31
0.0166166987366

0.0
i=  2
j=  148
0.0027129646618967193
i=  2
j=  149
0.006477625634683405
i=  2
j=  150
0.01639603739345703
i=  2
j=  151
0.0
i=  2
j=  152
0.01281460268740124
i=  2
j=  153
0.003993925462954924
i=  2
j=  154
0.021370297688913017
i=  2
j=  155
0.04960076986858568
i=  2
j=  156
0.0037636188252007084
i=  2
j=  157
0.006448996680288371
i=  2
j=  158
0.009401536574177841
i=  2
j=  159
0.02210830445709858
i=  2
j=  160
0.019853931848537437
i=  2
j=  161
0.011219436779755135
i=  2
j=  162
0.004051468318393216
i=  2
j=  163
0.05184749242079364
i=  2
j=  164
0.05184749242079364
i=  2
j=  165
0.0
i=  2
j=  166
0.0
i=  2
j=  167
0.0
i=  2
j=  168
0.027650392397304695
i=  2
j=  169
0.007378046744337688
i=  2
j=  170
0.004242416799238389
i=  2
j=  171
0.012315410824377886
i=  2
j=  172
0.0
i=  2
j=  173
0.0
i=  2
j=  174
0.0
i=  2
j=  175
0.00675557791625665
i=  2
j=  176
0.0
i=  2
j=  177
0.00675557791625665
i=  2
j=  178
0.0
i=  2
j=  179
0.014838899191236748
i=  2
j=  180
0.0
i=  2
j=  181
0.0

0.017979917268053064
i=  4
j=  114
0.011818548803110203
i=  4
j=  115
0.021642036429165977
i=  4
j=  116
0.0022655519812543657
i=  4
j=  117
0.004928177367641804
i=  4
j=  118
0.033092980889349484
i=  4
j=  119
0.0029191633731054417
i=  4
j=  120
0.021525928227425332
i=  4
j=  121
0.015152518007276178
i=  4
j=  122
0.08832224756461703
i=  4
j=  123
0.009054421851416783
i=  4
j=  124
0.01936163234710548
i=  4
j=  125
0.008668168444406004
i=  4
j=  126
0.0018646585527176246
i=  4
j=  127
0.026285734885052387
i=  4
j=  128
0.015285692219783042
i=  4
j=  129
0.11524682952021047
i=  4
j=  130
0.015083778081505302
i=  4
j=  131
0.04239349853696539
i=  4
j=  132
0.0016344816878604712
i=  4
j=  133
0.010402274562771447
i=  4
j=  134
0.010488092460904187
i=  4
j=  135
0.021826112489590686
i=  4
j=  136
0.06070003437345667
i=  4
j=  137
0.02409286906995005
i=  4
j=  138
0.002029437215033761
i=  4
j=  139
0.009616745701220753
i=  4
j=  140
1.0000000000000002
i=  4
j=  141
--------- 1.000000000000

0.01002590654585464
i=  6
j=  49
0.02892026403570741
i=  6
j=  50
0.005910215644562529
i=  6
j=  51
0.011323604524704553
i=  6
j=  52
0.013391020770065758
i=  6
j=  53
0.013229697620202905
i=  6
j=  54
0.009411879508845979
i=  6
j=  55
0.029897963607478036
i=  6
j=  56
0.009316798564125076
i=  6
j=  57
0.02998343317202206
i=  6
j=  58
0.005922651914236255
i=  6
j=  59
0.012235189134801754
i=  6
j=  60
0.02332751120462161
i=  6
j=  61
0.03987080657670749
i=  6
j=  62
0.004812674777686279
i=  6
j=  63
0.016793053105100816
i=  6
j=  64
0.05887855904814755
i=  6
j=  65
0.02173918802145484
i=  6
j=  66
0.016476446404701552
i=  6
j=  67
0.02258492757114465
i=  6
j=  68
0.019140291724124358
i=  6
j=  69
0.004359510046829472
i=  6
j=  70
0.0
i=  6
j=  71
0.031584464274931466
i=  6
j=  72
0.005055333580873284
i=  6
j=  73
0.010508324888645303
i=  6
j=  74
0.013019912823831001
i=  6
j=  75
0.02246429295158934
i=  6
j=  76
0.0030677933285705884
i=  6
j=  77
0.02258492757114465
i=  6
j=  78
0.0
i=

0.004744815694202918
i=  7
j=  190
0.0201888370282006
i=  7
j=  191
0.01803040596022567
i=  7
j=  192
0.028706741835551164
i=  7
j=  193
0.009316077901269353
i=  7
j=  194
0.051973804510132886
i=  7
j=  195
0.002005786294342831
i=  7
j=  196
0.0017362137273743154
i=  8
j=  1
0.00565505829153073
i=  8
j=  2
0.004514435183763644
i=  8
j=  3
0.009010896637109677
i=  8
j=  4
0.00984600456512107
i=  8
j=  5
0.0
i=  8
j=  6
0.028706741835551164
i=  8
j=  7
1.0000000000000002
i=  8
j=  8
0.031176150148311932
i=  8
j=  9
0.006525705211800851
i=  8
j=  10
0.013801173403984695
i=  8
j=  11
0.02834506520950804
i=  8
j=  12
0.033166454775768926
i=  8
j=  13
0.004634158287108645
i=  8
j=  14
0.022919168646779203
i=  8
j=  15
0.023298810883126596
i=  8
j=  16
0.006801432137695243
i=  8
j=  17
0.002380713651818842
i=  8
j=  18
0.017202210328840173
i=  8
j=  19
0.023339871655857023
i=  8
j=  20
0.001980554553752444
i=  8
j=  21
0.0030297067367022266
i=  8
j=  22
0.00695081341456976
i=  8
j=  23
0.0258

...

--------- 1.0
0.017614086350752282
i=  191
j=  125
0.005536092799529474
i=  191
j=  126
0.010680410617815108
i=  191
j=  127
0.026886243665042008
i=  191
j=  128
0.10078748850210473
i=  191
j=  129
0.01070665605542752
i=  191
j=  130
0.03293910257516919
i=  191
j=  131
0.010590081103117469
i=  191
j=  132
0.018448498781710423
i=  191
j=  133
0.016283135889227775
i=  191
j=  134
0.028665927892615302
i=  191
j=  135
0.028821038130195056
i=  191
j=  136
0.030819326791798135
i=  191
j=  137
0.026291647821762052
i=  191
j=  138
0.07182549900047187
i=  191
j=  139
0.01615350269925959
i=  191
j=  140
0.009054421851416783
i=  191
j=  141
0.014300350031273229
i=  191
j=  142
0.014464971002540442
i=  191
j=  143
0.014300350031273229
i=  191
j=  144
0.002276376702584787
i=  191
j=  145
0.008076944135182172
i=  191
j=  146
0.011137057188642586
i=  191
j=  147
0.02187141815602905
i=  191
j=  148
0.014963145875985876
i=  191
j=  149
0.01900007449240625
i=  191
j=  150
0.07182549900047187
i=  191
j= 

0.0
i=  192
j=  182
0.009492695196986846
i=  192
j=  183
0.03765937836119537
i=  192
j=  184
0.0
i=  192
j=  185
0.012915766644009861
i=  192
j=  186
0.00916606642652249
i=  192
j=  187
0.017429007659165514
i=  192
j=  188
0.07785743387503684
i=  192
j=  189
0.024374951306098777
i=  192
j=  190
0.0177558792064151
i=  192
j=  191
1.0000000000000002
i=  192
j=  192
0.008906403299245268
i=  192
j=  193
0.08272864019119405
i=  192
j=  194
0.05126934502751042
i=  192
j=  195
0.013720263736937738
i=  192
j=  196
0.0017362137273743154
i=  193
j=  1
0.00565505829153073
i=  193
j=  2
0.004514435183763644
i=  193
j=  3
0.009010896637109677
i=  193
j=  4
0.00984600456512107
i=  193
j=  5
0.0
i=  193
j=  6
0.028706741835551164
i=  193
j=  7
1.0000000000000002
i=  193
j=  8
--------- 1.0000000000000002
0.031176150148311932
i=  193
j=  9
0.006525705211800851
i=  193
j=  10
0.013801173403984695
i=  193
j=  11
0.02834506520950804
i=  193
j=  12
0.033166454775768926
i=  193
j=  13
0.004634158287108645


0.004478402482044694
i=  194
j=  44
0.03507136174773756
i=  194
j=  45
0.11888907873986478
i=  194
j=  46
0.020722193951907674
i=  194
j=  47
0.008435289526191398
i=  194
j=  48
0.0038606371438733283
i=  194
j=  49
0.011761645704342183
i=  194
j=  50
0.009236375648715846
i=  194
j=  51
0.011062249018085932
i=  194
j=  52
0.017944235996577095
i=  194
j=  53
0.004846561128767309
i=  194
j=  54
0.0
i=  194
j=  55
0.0026371175676815884
i=  194
j=  56
0.033820428786311175
i=  194
j=  57
0.049744114385194604
i=  194
j=  58
0.00946337485480599
i=  194
j=  59
0.008435289526191398
i=  194
j=  60
0.022370767170384456
i=  194
j=  61
0.0
i=  194
j=  62
0.0
i=  194
j=  63
0.0054548188562781996
i=  194
j=  64
0.0073711380686203335
i=  194
j=  65
0.00891434870700283
i=  194
j=  66
0.00160075655300736
i=  194
j=  67
0.01706654779286002
i=  194
j=  68
0.013981122071101703
i=  194
j=  69
0.009453755482739168
i=  194
j=  70
0.00498133146617816
i=  194
j=  71
0.0
i=  194
j=  72
0.06485370668543415
i=  194

0.002904779567976432
i=  195
j=  101
0.04637828677400365
i=  195
j=  102
0.017606810341495374
i=  195
j=  103
0.027328960875308218
i=  195
j=  104
0.0035391828888858903
i=  195
j=  105
0.007224815706193208
i=  195
j=  106
0.021221029500267455
i=  195
j=  107
0.029528543973207897
i=  195
j=  108
0.014966088287275733
i=  195
j=  109
0.01825296358913805
i=  195
j=  110
0.008526249447145855
i=  195
j=  111
0.10200322322831527
i=  195
j=  112
0.0493840820357265
i=  195
j=  113
0.00929203311844239
i=  195
j=  114
0.01985885708859687
i=  195
j=  115
0.027873675043122357
i=  195
j=  116
0.00495112180533849
i=  195
j=  117
0.018757776547904018
i=  195
j=  118
0.013614482062346953
i=  195
j=  119
0.012992613391338831
i=  195
j=  120
0.0052941032146333975
i=  195
j=  121
0.006972075631088926
i=  195
j=  122
0.018219876381070203
i=  195
j=  123
0.049650041350301245
i=  195
j=  124
0.019953755340583593
i=  195
j=  125
0.004492850298151864
i=  195
j=  126
0.021234920050152987
i=  195
j=  127
0.04756

0.006584777423593324
i=  196
j=  158
0.0
i=  196
j=  159
0.0
i=  196
j=  160
0.011220781496014963
i=  196
j=  161
0.015253823655002037
i=  196
j=  162
0.006053258424977726
i=  196
j=  163
0.006072934441446043
i=  196
j=  164
0.006072934441446043
i=  196
j=  165
0.0019463829357274482
i=  196
j=  166
0.018670183398032295
i=  196
j=  167
0.01576641810768127
i=  196
j=  168
0.0072258002780725685
i=  196
j=  169
0.009035008094003037
i=  196
j=  170
0.010262206848157072
i=  196
j=  171
0.008541262719361453
i=  196
j=  172
0.014545584624214864
i=  196
j=  173
0.003291181184809555
i=  196
j=  174
0.005659770003171749
i=  196
j=  175
0.005274076208747771
i=  196
j=  176
0.004430502383223901
i=  196
j=  177
0.005274076208747771
i=  196
j=  178
0.007524045378511892
i=  196
j=  179
0.008273119598793182
i=  196
j=  180
0.004430502383223901
i=  196
j=  181
0.006418145088438011
i=  196
j=  182
0.001219428484520887
i=  196
j=  183
0.007536028046058917
i=  196
j=  184
0.011434536614149303
i=  196
j=  1

In [133]:
a=df_train.judgment_text.iloc[20]

In [134]:
b=df_train.judgment_text.iloc[51]